In [73]:
import pandas as pd
import numpy as np

from datetime import datetime
from datetime import timedelta

<br>
<br>

Rows with index 14231, 11884 are manually corrected in the raw training data.

In [210]:
dateparse_1 = lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M')
dateparse_2 = lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

train_raw = pd.read_csv('train.csv', index_col = 0, parse_dates = ['start_time'], date_parser=dateparse_1)
test_raw = pd.read_csv('test.csv', index_col = 0, parse_dates = ['start_time'], date_parser=dateparse_2)

<br>
<br>





It is clear that var_bu, var_hl and end_time are useless. Drop them.

In [211]:
train_raw = train_raw.drop(['var_bu', 'var_hl', 'end_time'], axis = 1)
test_raw = test_raw.drop(['var_bu', 'var_hl', 'end_time'], axis = 1)

<br>
<br>


In order for ARIMA to handle seasonality well, plug in the missing rows, so that each hour has its own row.

In [212]:
train_raw.shape

(15919, 10)

In [213]:
test_raw.shape

(1460, 7)

<br>
<br>


Start from the training set

In [214]:
base_train = train_raw.loc[15919, 'start_time']
# 670 days * 24 hours = 16080
full_datetime_list_train = [base_train - timedelta(hours=x) for x in range(0, 16080)]

In [215]:
full_datetime_df_train = pd.DataFrame(full_datetime_list_train)
full_datetime_df_train.columns = ['start_time']
full_datetime_df_train = full_datetime_df_train.sort_values('start_time')

In [216]:
train_raw = train_raw.reset_index(drop = False)
train_expanded = pd.merge(full_datetime_df_train,
                          train_raw,
                          on='start_time', 
                          how='left')

<br>
<br>

Do the same for the test set

In [217]:
base_test = test_raw.loc[17379, 'start_time']
# 61 days * 24 hours = 1464
full_datetime_list_test = [base_test - timedelta(hours=x) for x in range(0, 1464)]

In [218]:
full_datetime_df_test = pd.DataFrame(full_datetime_list_test)
full_datetime_df_test.columns = ['start_time']
full_datetime_df_test = full_datetime_df_test.sort_values('start_time')

In [219]:
test_raw = test_raw.reset_index(drop = False)
test_expanded = pd.merge(full_datetime_df_test,
                         test_raw,
                         on='start_time', 
                         how='left')

<br>
<br>


Combine and then fill NA. Fill forward then backward instead of interpolation to avoid peeking into the future.

Notice that values to be predicted in the test dataset are temporarily filled anyway.

In [220]:
combined = pd.concat([train_expanded, test_expanded], axis=0, ignore_index = True, sort=False)
combined = combined.apply(lambda x: x.fillna(method = "ffill", axis = 0).\
                                      fillna(method = "bfill",axis = 0))

<br>
<br>

Remember previously there is a list of dataframes each is the daily data of a stock. 

The list_of_df_by_window is a list of dataframes each is a sliding window of 13 hours. 

As you will see in the later process, the first 12 hours will be features. For the last one hour, all other values will be discarded, except the "others" and "cust" which are what to be forecasted. 

The total number of dataframes in list_of_df_by_window should be: number of rows in dataset - window length + 1.

In [141]:
window_size = 13

def get_list_of_window_sized_df(df):
    return [df.iloc[i: i + window_size] for i in range(len(df.index) - window_size + 1)]

In [223]:
list_of_df_by_window = get_list_of_window_sized_df(combined)

<br>
<br>
An example of such windows.

In [246]:
list_of_df_by_window[0]

,start_time,idx,var_w,var_t,var_at,var_m,var_hm,var_wd,others,cust,total_cnt
0,2011-01-01 00:00:00,1.0,1000.0,119.68,68.79,17.0,40.5,0.00000,3.0,13.0,16.0
1,2011-01-01 01:00:00,2.0,1000.0,118.04,67.27,44.0,40.0,0.00000,8.0,32.0,40.0
2,2011-01-01 02:00:00,3.0,1000.0,118.04,67.27,44.0,40.0,0.00000,5.0,27.0,32.0
3,2011-01-01 03:00:00,4.0,1000.0,119.68,68.79,44.0,37.5,0.00000,3.0,10.0,13.0
4,2011-01-01 04:00:00,5.0,1000.0,119.68,68.79,44.0,37.5,0.00000,0.0,1.0,1.0
5,2011-01-01 05:00:00,6.0,2000.0,119.68,65.76,44.0,37.5,3.00160,0.0,1.0,1.0
6,2011-01-01 06:00:00,7.0,1000.0,118.04,67.27,44.0,40.0,0.00000,2.0,0.0,2.0
7,2011-01-01 07:00:00,8.0,1000.0,116.40,65.76,44.0,43.0,0.00000,1.0,2.0,3.0
8,2011-01-01 08:00:00,9.0,1000.0,119.68,68.79,44.0,37.5,0.00000,1.0,7.0,8.0
9,2011-01-01 09:00:00,10.0,1000.0,126.24,74.85,44.0,38.0,0.00000,8.0,6.0,14.0


<br>
<br>

### Transform to Tidy format

The next step is to transform each window into a row. The list_of_df_by_stock_by_window_flatten_reindexed_unstacked is a list of such rows.

In [230]:
list_of_df_by_window_reindexed_unstacked = \
    [window.reset_index().unstack()
     for window in list_of_df_by_window]

<br>
<br>
Concatenate rows into a dataframe.

In [233]:
tidy_table = pd.concat(list_of_df_by_window_reindexed_unstacked, axis=1).T

In [237]:
tidy_table.shape

(17532, 156)

In [267]:
tidy_table.head()

,index_0,index_1,index_2,index_3,index_4,index_5,index_6,index_7,index_8,index_9,...,total_cnt_3,total_cnt_4,total_cnt_5,total_cnt_6,total_cnt_7,total_cnt_8,total_cnt_9,total_cnt_10,total_cnt_11,total_cnt_12
0,0,1,2,3,4,5,6,7,8,9,...,13,1,1,2,3,8,14,36,56,84
1,1,2,3,4,5,6,7,8,9,10,...,1,1,2,3,8,14,36,56,84,94
2,2,3,4,5,6,7,8,9,10,11,...,1,2,3,8,14,36,56,84,94,106
3,3,4,5,6,7,8,9,10,11,12,...,2,3,8,14,36,56,84,94,106,110
4,4,5,6,7,8,9,10,11,12,13,...,3,8,14,36,56,84,94,106,110,93


In [269]:
tidy_table.tail()

,index_0,index_1,index_2,index_3,index_4,index_5,index_6,index_7,index_8,index_9,...,total_cnt_3,total_cnt_4,total_cnt_5,total_cnt_6,total_cnt_7,total_cnt_8,total_cnt_9,total_cnt_10,total_cnt_11,total_cnt_12
17527,17527,17528,17529,17530,17531,17532,17533,17534,17535,17536,...,101,101,101,101,101,101,101,101,101,101
17528,17528,17529,17530,17531,17532,17533,17534,17535,17536,17537,...,101,101,101,101,101,101,101,101,101,101
17529,17529,17530,17531,17532,17533,17534,17535,17536,17537,17538,...,101,101,101,101,101,101,101,101,101,101
17530,17530,17531,17532,17533,17534,17535,17536,17537,17538,17539,...,101,101,101,101,101,101,101,101,101,101
17531,17531,17532,17533,17534,17535,17536,17537,17538,17539,17540,...,101,101,101,101,101,101,101,101,101,101


<br>
<br>
Flatten the column names.

In [238]:
tidy_table.columns = [col[0] + '_' + str(col[1]) for col in tidy_table.columns.values]

<br>
<br>

### Feature engineering and train test data split

Drop useless columns.

In [264]:
tidy_table_dropped = \
tidy_table.drop(['index_0', 'index_1', 'index_2', 'index_3', 'index_4', 'index_5', 
                 'index_6', 'index_7', 'index_8', 'index_9', 'index_10', 'index_11',
                 'index_12',
                 'start_time_0', 'start_time_1', 'start_time_2', 'start_time_3', 
                 'start_time_4', 'start_time_5', 'start_time_6', 'start_time_7', 
                 'start_time_8', 'start_time_9', 'start_time_10', 'start_time_11', 
                 'idx_0', 'idx_1', 'idx_2', 'idx_3', 'idx_4', 'idx_5', 'idx_6', 
                 'idx_7', 'idx_8', 'idx_9', 'idx_10', 'idx_11', 
                 'others_0', 'others_1', 'others_2', 'others_3', 'others_4', 
                 'others_5', 'others_6', 'others_7', 'others_8', 'others_9', 
                 'others_10', 'others_11', 
                 'cust_0', 'cust_1', 'cust_2', 'cust_3', 'cust_4', 'cust_5', 
                 'cust_6', 'cust_7', 'cust_8', 'cust_9', 'cust_10', 'cust_11', 
                 'total_cnt_0', 'total_cnt_1', 'total_cnt_2', 'total_cnt_3', 
                 'total_cnt_4', 'total_cnt_5', 'total_cnt_6', 'total_cnt_7', 
                 'total_cnt_8', 'total_cnt_9', 'total_cnt_10', 'total_cnt_11'
                ], axis = 1)

In [266]:
tidy_table_dropped.head()

,start_time_12,idx_12,var_w_0,var_w_1,var_w_2,var_w_3,var_w_4,var_w_5,var_w_6,var_w_7,...,var_wd_6,var_wd_7,var_wd_8,var_wd_9,var_wd_10,var_wd_11,var_wd_12,others_12,cust_12,total_cnt_12
0,2011-01-01 12:00:00,13,1000,1000,1000,1000,1000,2000,1000,1000,...,0,0,0,0,8.49895,9.5006,9.5006,29,55,84
1,2011-01-01 13:00:00,14,1000,1000,1000,1000,2000,1000,1000,1000,...,0,0,0,8.49895,9.5006,9.5006,9.99975,47,47,94
2,2011-01-01 14:00:00,15,1000,1000,1000,2000,1000,1000,1000,1000,...,0,0,8.49895,9.5006,9.5006,9.99975,9.5006,35,71,106
3,2011-01-01 15:00:00,16,1000,1000,2000,1000,1000,1000,1000,1000,...,0,8.49895,9.5006,9.5006,9.99975,9.5006,9.99975,40,70,110
4,2011-01-01 16:00:00,17,1000,2000,1000,1000,1000,1000,1000,1000,...,8.49895,9.5006,9.5006,9.99975,9.5006,9.99975,9.99975,41,52,93


In [270]:
tidy_table_dropped.shape

(17532, 83)

What columns are left?

In [271]:
tidy_table_dropped.columns

Index(['start_time_12', 'idx_12', 'var_w_0', 'var_w_1', 'var_w_2', 'var_w_3',
       'var_w_4', 'var_w_5', 'var_w_6', 'var_w_7', 'var_w_8', 'var_w_9',
       'var_w_10', 'var_w_11', 'var_w_12', 'var_t_0', 'var_t_1', 'var_t_2',
       'var_t_3', 'var_t_4', 'var_t_5', 'var_t_6', 'var_t_7', 'var_t_8',
       'var_t_9', 'var_t_10', 'var_t_11', 'var_t_12', 'var_at_0', 'var_at_1',
       'var_at_2', 'var_at_3', 'var_at_4', 'var_at_5', 'var_at_6', 'var_at_7',
       'var_at_8', 'var_at_9', 'var_at_10', 'var_at_11', 'var_at_12',
       'var_m_0', 'var_m_1', 'var_m_2', 'var_m_3', 'var_m_4', 'var_m_5',
       'var_m_6', 'var_m_7', 'var_m_8', 'var_m_9', 'var_m_10', 'var_m_11',
       'var_m_12', 'var_hm_0', 'var_hm_1', 'var_hm_2', 'var_hm_3', 'var_hm_4',
       'var_hm_5', 'var_hm_6', 'var_hm_7', 'var_hm_8', 'var_hm_9', 'var_hm_10',
       'var_hm_11', 'var_hm_12', 'var_wd_0', 'var_wd_1', 'var_wd_2',
       'var_wd_3', 'var_wd_4', 'var_wd_5', 'var_wd_6', 'var_wd_7', 'var_wd_8',
       'var_wd_9',

<br>
<br>
Define a function to generate more features. Specifically:

- Rowing sum of 2-hour window size

- Rowing sum of 4-hour window size

- Rowing sum of 8-hour window size

- Rowing sum of 12-hour window size

In [ ]:
def add_more_rolling_features(df, string):
    df[string + "_0_1"] = df[string + "_0"] + df[string + "_1"]
    df[string + "_1_2"] = df[string + "_1"] + df[string + "_2"]
    df[string + "_2_3"] = df[string + "_2"] + df[string + "_3"]
    df[string + "_3_4"] = df[string + "_3"] + df[string + "_4"]
    df[string + "_4_5"] = df[string + "_4"] + df[string + "_5"]
    df[string + "_5_6"] = df[string + "_5"] + df[string + "_6"]
    df[string + "_6_7"] = df[string + "_6"] + df[string + "_7"]
    df[string + "_7_8"] = df[string + "_7"] + df[string + "_8"]
    df[string + "_8_9"] = df[string + "_8"] + df[string + "_9"]
    df[string + "_9_10"] = df[string + "_9"] + df[string + "_10"]
    df[string + "_10_11"] = df[string + "_10"] + df[string + "_11"]
    
    df[string + "_0_to_3"] = df[string + "_0"] + df[string + "_1"] + df[string + "_2"] + df[string + "_3"]
    df[string + "_1_to_4"] = df[string + "_1"] + df[string + "_2"] + df[string + "_3"] + df[string + "_4"]
    df[string + "_2_to_5"] = df[string + "_2"] + df[string + "_3"] + df[string + "_4"] + df[string + "_5"]
    df[string + "_3_to_6"] = df[string + "_3"] + df[string + "_4"] + df[string + "_5"] + df[string + "_6"]
    df[string + "_4_to_7"] = df[string + "_4"] + df[string + "_5"] + df[string + "_6"] + df[string + "_7"]
    df[string + "_5_to_8"] = df[string + "_5"] + df[string + "_6"] + df[string + "_7"] + df[string + "_8"]
    df[string + "_6_to_9"] = df[string + "_6"] + df[string + "_7"] + df[string + "_8"] + df[string + "_9"]
    df[string + "_7_to_10"] = df[string + "_7"] + df[string + "_8"] + df[string + "_9"] + \
                              df[string + "_10"]
    df[string + "_8_to_11"] = df[string + "_8"] + df[string + "_9"] + df[string + "_10"] + \
                              df[string + "_11"]
    
    df[string + "_0_to_7"] = df[string + "_0"] + df[string + "_1"] + df[string + "_2"] + \
                             df[string + "_3"] + df[string + "_4"] + df[string + "_5"] + \
                             df[string + "_6"] + df[string + "_7"]
    df[string + "_1_to_8"] = df[string + "_1"] + df[string + "_2"] + df[string + "_3"] + \
                             df[string + "_4"] + df[string + "_5"] + df[string + "_6"] + \
                             df[string + "_7"] + df[string + "_8"]
    df[string + "_2_to_9"] = df[string + "_2"] + df[string + "_3"] + df[string + "_4"] + \
                             df[string + "_5"] + df[string + "_6"] + df[string + "_7"] + \
                             df[string + "_8"] + df[string + "_9"]
    df[string + "_3_to_10"] = df[string + "_3"] + df[string + "_4"] + df[string + "_5"] + \
                              df[string + "_6"] + df[string + "_7"] + df[string + "_8"] + \
                              df[string + "_9"] + df[string + "_10"]
    df[string + "_4_to_11"] = df[string + "_4"] + df[string + "_5"] + df[string + "_6"] + \
                              df[string + "_7"] + df[string + "_8"] + df[string + "_9"] + \
                              df[string + "_10"] + df[string + "_11"]
    
    df[string + "_0_to_11"] = df[string + "_0"] + df[string + "_1"] + df[string + "_2"] + \
                              df[string + "_3"] + df[string + "_4"] + df[string + "_5"] + \
                              df[string + "_6"] + df[string + "_7"] + df[string + "_8"] + \
                              df[string + "_9"] + df[string + "_10"] + df[string + "_11"]
    return df

<br>
<br>
Apply this feature generation function to all useful features. 

In [ ]:
tidy_more_features = add_more_rolling_features(tidy_more_features, "var_w")
tidy_more_features = add_more_rolling_features(tidy_more_features, "volume")
tidy_more_features = add_more_rolling_features(tidy_more_features, "volatility")
tidy_more_features = add_more_rolling_features(tidy_more_features, "signal_A")
tidy_more_features = add_more_rolling_features(tidy_more_features, "signal_B")